In [1]:
# wandb and pytorch manual 
# https://colab.research.google.com/drive/1XDtq-KT0GkX06a_g1MevuLFOMk4TxjKZ#scrollTo=bZpt5W2NNl6S

# for hyperparameter sweeping
# https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=W_YSZCkITmVJ


In [2]:
from __future__ import print_function
import os
import argparse
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torchvision import datasets, transforms

import logging
logging.propagate = False
logging.getLogger().setLevel(logging.ERROR)

import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime
import wandb

from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

import sys
sys.path.append('../')

In [3]:
!wandb login

wandb: Currently logged in as: beomgon-yu. Use `wandb login --relogin` to force relogin


## hyperparameter sweep using wandb

In [4]:
# use random search
sweep_config = {
    'method': 'random'
    }

# for bayesian search, this value should be included
metric = {
    'name': 'f1_score',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric    

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        # 'values': ['adam']
        },
    }

parameters_dict.update({
    'epochs': {
        'value': 100}
    })

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'lr': {
        # a q_log_uniform_values distribution between 0 and 0.1
        'distribution': 'q_log_uniform_values',
        'q': 1e-5,
        'min': 1e-4,
        'max': 5e-2
      },
    'batch_size': {
        'values': [128,256,512,1024]
      },
    'svd_reduction': {
        'values': [True, False]
      },    
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'f1_score'},
 'parameters': {'batch_size': {'values': [128, 256, 512, 1024]},
                'epochs': {'value': 100},
                'lr': {'distribution': 'q_log_uniform_values',
                       'max': 0.05,
                       'min': 0.0001,
                       'q': 1e-05},
                'optimizer': {'values': ['adam', 'sgd']},
                'svd_reduction': {'values': [True, False]}}}


In [6]:
sweep_id = wandb.sweep(sweep_config, project="Credit Card Fraud Detection Sweep")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ep2s3sgj
Sweep URL: https://wandb.ai/beomgon-yu/Credit%20Card%20Fraud%20Detection%20Sweep/sweeps/ep2s3sgj


# set seed for reproduce

In [7]:
from train import seed_everything

## For SVD reduction

In [9]:
def reduced_mat(df) :
    u, s, v = np.linalg.svd(df, full_matrices=False)
    s[0] = 0
    rdf = pd.DataFrame( u @ np.diag(s) @ v)
    return rdf

# dataset

In [10]:
from dataset import CDataset

## model

In [12]:
from models import AutoEncoder, TiedAE, StackedAE

In [14]:
class Trainer() :
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, config) :
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = config.device
        self.epochs = config.epochs
        self.lr = config.lr
        
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self,) :
        self.model.to(self.device)
        best_score = 0
        for epoch in range(self.epochs) :
            self.model.train()
            train_loss = []
            
            for x in iter(self.train_loader) :
                x = x.to(self.device)
                _x = self.model(x)
                
                loss = self.criterion(x, _x)
                # loss = self.criterion(x, _x) + 0.5*self.criterion(x, x1)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
                wandb.log({'train loss' : loss.item()})
                
            score = self.validation(self.model, 0.95)
            wandb.log({'f1_score' : score})
            
            if self.scheduler is not None :
                self.scheduler.step(score)

            
    def validation(self, eval_model, threshold) :
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred_y = []
        true_y = []
        
        with torch.no_grad():
            for x, y in iter(self.val_loader) :
                x = x.to(self.device)
                y = y.to(self.device)
                
                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff) < threshold, 1, 0).tolist()
                pred_y += batch_pred
                true_y += y.tolist()

        return f1_score(true_y, pred_y, average='macro')

In [15]:

def main() :
    
    with wandb.init(config=None) :
        
        config = wandb.config
        config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        config.num_workers = 8
        config.seed = 42
        config.momentum = 0.9
        config.weight_decay = 1e-4
        
        seed_everything(config.seed) 

        train_df = pd.read_csv('../dataset/train.csv')
        val_df = pd.read_csv('../dataset/val.csv')
        train_df = train_df.drop(columns=['ID'])
        val_df = val_df.drop(columns=['ID'])  
        
        if config.svd_reduction :
            train_df = reduced_mat(train_df)
            val = reduced_mat(val_df.drop(columns=['Class']))
            val['Class'] = val_df['Class']
            val_df = val

        train_dataset = CDataset(train_df)
        train_loader = DataLoader(train_dataset, batch_size = config.batch_size, shuffle=True, num_workers=config.num_workers)

        val_dataset = CDataset(val_df, eval_mode=True)
        val_loader = DataLoader(val_dataset, batch_size = config.batch_size, shuffle=False, num_workers=config.num_workers)    

        model = AutoEncoder()
        model.eval()

        # if config.adam :
        if config.optimizer == 'adam' :
            optimizer = torch.optim.Adam(params=model.parameters(), lr = config.lr)
        else :
            optimizer = torch.optim.SGD(model.parameters(), config.lr,
                                        momentum=config.momentum,
                                        weight_decay=config.weight_decay)

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
        # scheduler = StepLR(optimizer, step_size=50, gamma=0.2)

        wandb.watch(model, log='all')

        trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, config)

        wandb.save('model.h5')

        trainer.fit()

In [ ]:
wandb.agent(sweep_id, main, count=20)

wandb: Agent Starting Run: m7q17neo with config:
wandb: 	batch_size: 512
wandb: 	epochs: 100
wandb: 	lr: 0.013970000000000002
wandb: 	optimizer: sgd
wandb: 	svd_reduction: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: beomgon-yu. Use `wandb login --relogin` to force relogin
